In [1]:
from model import wrn_28_2
import torch
from tqdm import tqdm

/opt/conda/envs/usb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
m = wrn_28_2(pretrained=False,pretrained_path=None ,num_classes=10)

In [3]:
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# m = Net()

In [4]:
for ind,x in enumerate(m.children()):
    print(ind,x)

0 Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1 NetworkBlock(
  (layer): Sequential(
    (0): BasicBlock(
      (bn1): BatchNorm2d(16, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
      (relu1): LeakyReLU(negative_slope=0.1)
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
      (relu2): LeakyReLU(negative_slope=0.1)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convShortcut): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): BasicBlock(
      (bn1): BatchNorm2d(32, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
      (relu1): LeakyReLU(negative_slope=0.1)
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
      (relu2): L

In [5]:
# !pip install torch

In [6]:
from semilearn import get_dataset, get_data_loader, get_net_builder, get_algorithm, get_config, Trainer

In [7]:
config = {
    'algorithm': 'fixmatch',
    'net': 'wrn_28_2',
    'use_pretrain': False,  # todo: add pretrain
    'pretrain_path': None,

    # optimization configs
    'epoch': 3,
    'num_train_iter': 150,
    'num_eval_iter': 50,
    'optim': 'SGD',
    'lr': 0.03,
    'momentum': 0.9,
    'batch_size': 32,
    'eval_batch_size': 64,

    # dataset configs
    'dataset': 'cifar10',
    'num_labels': 40,
    'num_classes': 10,
    'img_size': 32,
    'crop_ratio': 0.875,
    'data_dir': './data',

    # algorithm specific configs
    'hard_label': True,
    'uratio': 3,
    'ulb_loss_ratio': 1.0,

    # device configs
    'gpu': 0,
    'world_size': 1,
    'distributed': False,
}
config = get_config(config)

/bin/sh: 1: netstat: not found


In [8]:
vars(config)

{'save_dir': './saved_models',
 'save_name': 'fixmatch',
 'resume': False,
 'load_path': None,
 'overwrite': True,
 'use_tensorboard': False,
 'epoch': 3,
 'num_train_iter': 150,
 'num_warmup_iter': 0,
 'num_eval_iter': 50,
 'num_log_iter': 5,
 'num_labels': 40,
 'batch_size': 32,
 'uratio': 3,
 'eval_batch_size': 64,
 'ema_m': 0.999,
 'ulb_loss_ratio': 1.0,
 'optim': 'SGD',
 'lr': 0.03,
 'momentum': 0.9,
 'weight_decay': 0.0005,
 'layer_decay': 0.75,
 'net': 'wrn_28_2',
 'net_from_name': False,
 'use_pretrain': False,
 'pretrain_path': None,
 'algorithm': 'fixmatch',
 'use_cat': False,
 'use_amp': False,
 'clip_grad': 0,
 'imb_algorithm': None,
 'data_dir': './data',
 'dataset': 'cifar10',
 'num_classes': 10,
 'train_sampler': 'RandomSampler',
 'num_workers': 1,
 'lb_imb_ratio': 1,
 'ulb_imb_ratio': 1,
 'ulb_num_labels': None,
 'img_size': 32,
 'crop_ratio': 0.875,
 'max_length': 512,
 'max_length_seconds': 4.0,
 'sample_rate': 16000,
 'world_size': 1,
 'rank': 0,
 'dist_url': 'tcp://

In [9]:
dataset_dict = get_dataset(config, config.algorithm, config.dataset, config.num_labels, config.num_classes, data_dir=config.data_dir)
train_lb_loader = get_data_loader(config, dataset_dict['train_lb'], config.batch_size)

Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


In [10]:
from semilearn.datasets.cv_datasets import get_cifar
import argparse

In [11]:
args_d = {'dataset': 'cifar10',
         'num_classes': 10,
         'train_sampler': 'RandomSampler',
         'num_workers': 1,
         'lb_imb_ratio': 1,
         'ulb_imb_ratio': 1,
          'batch_size': 1,
         'ulb_num_labels': None,
         'img_size': 32,
         'crop_ratio': 0.875,
         'num_labels': 40,
         'seed': 1,
         'epoch': 3,
         'num_train_iter':150,
         'net': 'wrn_28_8',
         'optim': 'SGD',
         'lr': 0.03,
         'momentum': 0.9,
         'weight_decay': 0.0005,
         'layer_decay': 0.75,
          'num_warmup_iter': 0}

parser = argparse.ArgumentParser(description='Semi-Supervised Learning (USB semilearn package)')
args = parser.parse_args("")
# args
for k in args_d:
        setattr(args, k, args_d[k])
lb_dset, ulb_dset, eval_dset = get_cifar(args,
          alg=None, 
          name='cifar10',
          num_labels=40,
          num_classes=10,
          data_dir='./data',
          include_lb_to_ulb=True)

Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


In [12]:
lb_dset[0]

{'idx_lb': 0,
 'x_lb': tensor([[[-1.1247, -1.1589, -1.1247,  ..., -1.0390, -1.0733, -1.1075],
          [-1.0904, -1.1247, -1.0904,  ..., -1.0562, -1.0904, -1.1418],
          [-1.1247, -1.1589, -1.1247,  ..., -1.0390, -1.0733, -1.1075],
          ...,
          [ 0.3652,  0.3309,  0.3652,  ...,  0.5022,  0.3481,  0.0912],
          [ 0.3652,  0.3652,  0.3652,  ...,  0.5022,  0.3309, -0.1657],
          [ 0.4337,  0.3652,  0.4337,  ...,  0.6563,  0.4851, -0.0458]],
 
         [[-0.8452, -0.8803, -0.8452,  ..., -0.7577, -0.7927, -0.8277],
          [-0.8803, -0.8978, -0.8803,  ..., -0.7752, -0.8102, -0.8627],
          [-0.8452, -0.8803, -0.8452,  ..., -0.7577, -0.7927, -0.8277],
          ...,
          [ 0.4153,  0.3803,  0.4153,  ...,  0.4503,  0.3803,  0.1877],
          [ 0.4153,  0.3978,  0.4153,  ...,  0.4853,  0.3803, -0.0924],
          [ 0.4853,  0.4153,  0.4853,  ...,  0.6429,  0.5028, -0.0574]],
 
         [[-0.2010, -0.2881, -0.2010,  ..., -0.0615, -0.0964, -0.1312],
      

In [13]:
train_lb_loader = get_data_loader(args, dataset_dict['train_lb'], config.batch_size)
train_ulb_loader = get_data_loader(args, dataset_dict['train_ulb'], int(config.batch_size * config.uratio))
eval_loader = get_data_loader(args, dataset_dict['eval'], config.eval_batch_size)

In [14]:
for x in train_lb_loader:
    print(x.keys())
    break

dict_keys(['idx_lb', 'x_lb', 'y_lb'])


In [15]:
for data_lb, data_ulb in zip(train_lb_loader, train_ulb_loader):
    print(data_lb.keys(),data_ulb.keys())
    break

dict_keys(['idx_lb', 'x_lb', 'y_lb']) dict_keys(['idx_ulb', 'x_ulb_w', 'x_ulb_s'])


In [16]:
from semilearn.core.utils import get_dataset, get_data_loader, get_optimizer, get_cosine_schedule_with_warmup

In [17]:
optimizer = get_optimizer(m, args.optim, args.lr, args.momentum, args.weight_decay, args.layer_decay)
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            args.num_train_iter,
                                            num_warmup_steps=args.num_warmup_iter)

In [18]:
!nvidia-smi

Fri Sep 30 23:50:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1A.0 Off |                    0 |
| N/A   41C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
# optimizer

In [20]:
## Train

In [21]:
torch.device('cuda')

device(type='cuda')

In [22]:
data_lb['x_lb'].shape

torch.Size([32, 3, 32, 32])

In [23]:
!export CUDA_LAUNCH_BLOCKING=1

In [24]:
device = torch.device('cuda:0')
print(device)


cuda:0


In [28]:
loss_ce = torch.nn.CrossEntropyLoss()

In [40]:

m.to(device)
m.train()
for _ in range(10):
    for ind,(data_lb, data_ulb) in tqdm(enumerate(zip(train_lb_loader, train_ulb_loader)),total=len(train_lb_loader)):
        x = data_lb['x_lb'].to(device)
        y = data_lb['y_lb'].to(device)
        optimizer.zero_grad()

        out = m(x)
        out = out['logits']
        loss = loss_ce(out,y)

        loss.backward()
        optimizer.step()
        scheduler.step()
        if ind%10==0:
            with torch.no_grad():
                print(loss.item())
    

  6%|▌         | 3/50 [00:00<00:06,  7.69it/s]

0.35790538787841797


 28%|██▊       | 14/50 [00:01<00:03, 11.42it/s]

0.17122308909893036


 48%|████▊     | 24/50 [00:02<00:02, 11.99it/s]

...

 66%|██████▌   | 33/50 [00:02<00:01, 12.46it/s]

0.015095085836946964


 86%|████████▌ | 43/50 [00:03<00:00, 12.45it/s]

0.02013203874230385


100%|██████████| 50/50 [00:04<00:00, 11.64it/s]


In [41]:
# !python -m torch.utils.collect_env

In [42]:
def predict(model,data_loader,device, use_ema_model=False, return_gt=False):
        model.to(device)
        model.eval()

        if use_ema_model:
            self.algorithm.ema.apply_shadow()

        y_true = []
        y_pred = []
        y_logits = []
        with torch.no_grad():
            for data in tqdm(data_loader,total=len(data_loader)):
                x = data['x_lb']
                y = data['y_lb']

                if isinstance(x, dict):
                    x = {k: v.cuda(device) for k, v in x.items()}
                else:
                    x = x.cuda(device)
                y = y.cuda(device)

                logits = model(x)['logits']
                    
                y_true.extend(y.cpu().tolist())
                y_pred.extend(torch.max(logits, dim=-1)[1].cpu().tolist())
                y_logits.append(torch.softmax(logits, dim=-1).cpu().numpy())
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        y_logits = np.concatenate(y_logits)

        if use_ema_model:
            self.algorithm.ema.restore()
        model.train()
        
        if return_gt:
            return y_pred, y_logits, y_true
        else:
            return y_pred, y_logits

In [43]:
print(device)

cuda:0


In [44]:
import numpy as np

In [45]:
y_pred, y_logits, y_true = predict(m,eval_loader,device,return_gt=True)

100%|██████████| 50/50 [00:02<00:00, 24.10it/s]


In [46]:
## Metrics

In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [48]:
# y_true = data_lb['y_lb']
# y_pred = data_lb['y_lb']
top1 = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
cf_mat = confusion_matrix(y_true, y_pred, normalize='true')

In [49]:
top1, precision, recall, f1, cf_mat

(0.1375,
 0.1991554842591709,
 0.13501015820563506,
 0.07389710276788139,
 array([[0.00319489, 0.46325879, 0.00638978, 0.03514377, 0.00319489,
         0.00319489, 0.00319489, 0.01916933, 0.00319489, 0.4600639 ],
        [0.        , 0.82568807, 0.        , 0.0030581 , 0.        ,
         0.        , 0.        , 0.0030581 , 0.        , 0.16819572],
        [0.        , 0.46031746, 0.0031746 , 0.15238095, 0.        ,
         0.00634921, 0.00952381, 0.2       , 0.        , 0.16825397],
        [0.        , 0.60120846, 0.00302115, 0.10876133, 0.        ,
         0.01812689, 0.0060423 , 0.13897281, 0.        , 0.12386707],
        [0.        , 0.35737705, 0.04262295, 0.19672131, 0.        ,
         0.01639344, 0.03278689, 0.26557377, 0.        , 0.08852459],
        [0.        , 0.50798722, 0.00638978, 0.12460064, 0.        ,
         0.02236422, 0.02555911, 0.11182109, 0.        , 0.20127796],
        [0.        , 0.42628205, 0.01282051, 0.13782051, 0.        ,
         0.        , 0.